In [1]:
#working
import cv2

# Load the pre-trained Haarcascades for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Function to detect eyes in a given frame
def detect_eyes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        if len(eyes) >= 2:
            return True  # Eyes detected

    return False  # Eyes not detected

# Open a video capture object
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Check if eyes are looking at the camera
    if detect_eyes(frame):
        cv2.putText(frame, "Eyes looking at the camera", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        cv2.putText(frame, "Eyes not looking at the camera", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


In [3]:
!pip install streamlit

     ---------------------------------------- 4.8/4.8 MB 1.3 MB/s eta 0:00:00
     -------------------------------------- 341.8/341.8 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 8.4/8.4 MB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 190.6/190.6 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 24.6/24.6 MB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 62.7/62.7 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1


In [6]:
import streamlit as st

from camera_input_live import camera_input_live

image = camera_input_live()

st.image(value)

ModuleNotFoundError: No module named 'camera_input_live'

In [7]:
!pip install cmake

   ---------------------------------------- 34.6/34.6 MB 2.6 MB/s eta 0:00:00


In [10]:
!pip install dlib-19.22.99-cp38-cp38-win_amd64.whl

ERROR: dlib-19.22.99-cp38-cp38-win_amd64.whl is not a supported wheel on this platform.


In [10]:
#working
import cv2
import dlib
import pyttsx3
import numpy as np

def calculate_gaze_ratio(eye_landmarks, landmarks):
    left_eye_region = np.array([(landmarks.part(eye_landmarks[0]).x, landmarks.part(eye_landmarks[0]).y),
                                (landmarks.part(eye_landmarks[1]).x, landmarks.part(eye_landmarks[1]).y),
                                (landmarks.part(eye_landmarks[2]).x, landmarks.part(eye_landmarks[2]).y),
                                (landmarks.part(eye_landmarks[3]).x, landmarks.part(eye_landmarks[3]).y),
                                (landmarks.part(eye_landmarks[4]).x, landmarks.part(eye_landmarks[4]).y),
                                (landmarks.part(eye_landmarks[5]).x, landmarks.part(eye_landmarks[5]).y)], np.int32)

    height, width, _ = frame.shape
    mask = np.zeros((height, width), np.uint8)
    cv2.polylines(mask, [left_eye_region], True, 255, 2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)

    min_x = np.min(left_eye_region[:, 0])
    max_x = np.max(left_eye_region[:, 0])
    min_y = np.min(left_eye_region[:, 1])
    max_y = np.max(left_eye_region[:, 1])

    gray_eye = eye[min_y:max_y, min_x:max_x]
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    height, width = threshold_eye.shape

    left_side_threshold = threshold_eye[0:height, 0:int(width / 2)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0:height, int(width / 2):width]
    right_side_white = cv2.countNonZero(right_side_threshold)

    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else:
        gaze_ratio = left_side_white / right_side_white

    return gaze_ratio

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)

        left_eye_ratio = calculate_gaze_ratio([ 36, 37, 38, 39, 40, 41], landmarks)

        if left_eye_ratio > 4:
            cv2.putText(frame, "ALERT!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            speak("Driver is using the mobile phone!")

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()


In [9]:
#working
import cv2
import dlib
import numpy as np
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Load pre-trained face and eye detectors
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize Dlib's face detector (HOG-based) and create predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to calculate the Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    # Compute the euclidean distances between the two sets of vertical eye landmarks
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])

    # Compute the euclidean distance between the horizontal eye landmarks
    C = np.linalg.norm(eye[0] - eye[3])

    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

# Start capturing video from the default camera (you can change the argument to use a different camera)
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = detector(gray)

    for face in faces:
        # Get the facial landmarks
        shape = predictor(gray, face)
        shape = np.array([[shape.part(i).x, shape.part(i).y] for i in range(68)])

        # Extract the left and right eye coordinates
        left_eye = shape[42:48]
        right_eye = shape[36:42]

        # Calculate the Eye Aspect Ratio (EAR) for each eye
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)

        # Calculate the average EAR
        ear = (left_ear + right_ear) / 2.0

        # Draw the facial landmarks on the frame
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)

        # Check if the driver is looking at the camera
        if ear > 0.1:
            cv2.putText(frame, "Driver is looking at the camera", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Driver is not looking at the camera", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            # Give a voice alarm
            engine.say("Please do not use a mobile device while driving.")
            engine.runAndWait()

    # Display the frame
    cv2.imshow('Driver Monitoring System', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


# Yawn


In [9]:
#working alhamdullah
import cv2
import dlib
import pyttsx3
import numpy as np
from scipy.spatial import distance as dist
import time

def calculate_mouth_openness(mouth_landmarks):
    upper_lip_top = np.mean(mouth_landmarks[13:16], axis=0)
    lower_lip_bottom = np.mean(mouth_landmarks[19:], axis=0)
    distance = dist.euclidean(upper_lip_top, lower_lip_bottom)
    return distance

# def speak(text):
#     engine = pyttsx3.init()
#     engine.say(text)
#     engine.runAndWait()
def play_sound(filename):
    pygame.mixer.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()


cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

start_time = None
yawn_duration_threshold = 2  # in seconds

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)

        
        mouth_landmarks = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(48, 68)], dtype=np.int32)

        
        mouth_openness = calculate_mouth_openness(mouth_landmarks)

        if mouth_openness > 14:  # Adjust the threshold based on your testing
            if start_time is None:
                start_time = time.time()
            else:
                elapsed_time = time.time() - start_time
                if elapsed_time > yawn_duration_threshold:
                    cv2.putText(frame, "ALERT! Yawning Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    play_sound('alarm_1.wav')
                    #                     speak("Driver is yawning!")

        else:
            start_time = None  

        # Draw mouth landmarks on the frame for visualization
#         cv2.polylines(frame, [mouth_landmarks], isClosed=True, color=(0, 255, 0), thickness=2)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:  
        break

cap.release()
cv2.destroyAllWindows()


# looking_at_the_camera

In [11]:
#working
import cv2
import dlib
import pyttsx3
import numpy as np
import time

def calculate_gaze_ratio(eye_landmarks, landmarks):
    left_eye_region = np.array([(landmarks.part(eye_landmarks[0]).x, landmarks.part(eye_landmarks[0]).y),
                                (landmarks.part(eye_landmarks[1]).x, landmarks.part(eye_landmarks[1]).y),
                                (landmarks.part(eye_landmarks[2]).x, landmarks.part(eye_landmarks[2]).y),
                                (landmarks.part(eye_landmarks[3]).x, landmarks.part(eye_landmarks[3]).y),
                                (landmarks.part(eye_landmarks[4]).x, landmarks.part(eye_landmarks[4]).y),
                                (landmarks.part(eye_landmarks[5]).x, landmarks.part(eye_landmarks[5]).y)], np.int32)

    height, width, _ = frame.shape
    mask = np.zeros((height, width), np.uint8)
    cv2.polylines(mask, [left_eye_region], True, 255, 2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)

    min_x = np.min(left_eye_region[:, 0])
    max_x = np.max(left_eye_region[:, 0])
    min_y = np.min(left_eye_region[:, 1])
    max_y = np.max(left_eye_region[:, 1])

    gray_eye = eye[min_y:max_y, min_x:max_x]
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    height, width = threshold_eye.shape

    left_side_threshold = threshold_eye[0:height, 0:int(width / 6)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0:height, int(width / 6):width]  # Fix indexing here
    right_side_white = cv2.countNonZero(right_side_threshold)
    



    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else:
        gaze_ratio = left_side_white / right_side_white

    return gaze_ratio

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

start_time = None
look_duration_threshold = 2  # in seconds

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)

        left_eye_ratio = calculate_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
        if left_eye_ratio > 4 or left_eye_ratio < 0.25: 
#         if left_eye_ratio > 4:
            if start_time is None:
                start_time = time.time()
            else:
                elapsed_time = time.time() - start_time
                if elapsed_time > look_duration_threshold:
                    cv2.putText(frame, "ALERT!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    speak("Driver is looking at the camera for too long!")

        else:
            start_time = None  # Reset the timer when the driver is not looking at the camera

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()


# closed eyes 

In [1]:
#working
import cv2
import dlib
import pyttsx3
import numpy as np
import time

def calculate_eye_aspect_ratio(eye_landmarks):
    vertical_dist1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    vertical_dist2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])

    horizontal_dist = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])

    aspect_ratio = (vertical_dist1 + vertical_dist2) / (2.0 * horizontal_dist)
    
    return aspect_ratio

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

left_eye_landmarks = [36, 37, 38, 39, 40, 41]
right_eye_landmarks = [42, 43, 44, 45, 46, 47]

start_time_left = None
start_time_right = None
eye_closed_duration_threshold = 2 

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)

        left_eye_points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in left_eye_landmarks], dtype=np.int32)
        aspect_ratio_left = calculate_eye_aspect_ratio(left_eye_points)

        right_eye_points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in right_eye_landmarks], dtype=np.int32)
        aspect_ratio_right = calculate_eye_aspect_ratio(right_eye_points)

        if aspect_ratio_left < 0.2 or aspect_ratio_right < 0.2:  # Adjust the threshold based on your testing
            if aspect_ratio_left < 0.2 and start_time_left is None:
                start_time_left = time.time()
            elif aspect_ratio_right < 0.2 and start_time_right is None:
                start_time_right = time.time()
            else:
                elapsed_time_left = time.time() - start_time_left if start_time_left is not None else 0
                elapsed_time_right = time.time() - start_time_right if start_time_right is not None else 0

                if elapsed_time_left > eye_closed_duration_threshold or elapsed_time_right > eye_closed_duration_threshold:
                    cv2.putText(frame, "ALERT! Eyes Closed", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    speak("Wake up!")

        else:
            start_time_left = None
            start_time_right = None

#         cv2.polylines(frame, [left_eye_points], isClosed=True, color=(0, 255, 0), thickness=2)
#         cv2.polylines(frame, [right_eye_points], isClosed=True, color=(0, 255, 0), thickness=2)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:  
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# [0:height, int(width / 2):width]  # Fix indexing here

In [7]:
# !pip install pygame


  Obtaining dependency information for pygame from https://files.pythonhosted.org/packages/82/61/93ae7afbd931a70510cfdf0a7bb0007540020b8d80bc1d8762ebdc46479b/pygame-2.5.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB 131.3 kB/s eta 0:01:22
   ---------------------------------------- 0.0/10.8 MB 131.3 kB/s eta 0:01:22
   ---------------------------------------- 0.0/10.8 MB 131.3 kB/s eta 0:01:

# sleeping with alarm

In [10]:
import cv2
import dlib
import numpy as np
import time
import pygame

def calculate_eye_aspect_ratio(eye_landmarks):
    vertical_dist1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    vertical_dist2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])

    horizontal_dist = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])

    aspect_ratio = (vertical_dist1 + vertical_dist2) / (2.0 * horizontal_dist)
    
    return aspect_ratio

def play_sound(filename):
    pygame.mixer.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

left_eye_landmarks = [36, 37, 38, 39, 40, 41]
right_eye_landmarks = [42, 43, 44, 45, 46, 47]

start_time_left = None
start_time_right = None
eye_closed_duration_threshold = 2  

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)

       
        left_eye_points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in left_eye_landmarks], dtype=np.int32)
        aspect_ratio_left = calculate_eye_aspect_ratio(left_eye_points)

        
        right_eye_points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in right_eye_landmarks], dtype=np.int32)
        aspect_ratio_right = calculate_eye_aspect_ratio(right_eye_points)

        
        if aspect_ratio_left < 0.2 or aspect_ratio_right < 0.2:  
            if aspect_ratio_left < 0.2 and start_time_left is None:
                start_time_left = time.time()
            elif aspect_ratio_right < 0.2 and start_time_right is None:
                start_time_right = time.time()
            else:
                elapsed_time_left = time.time() - start_time_left if start_time_left is not None else 0
                elapsed_time_right = time.time() - start_time_right if start_time_right is not None else 0

                if elapsed_time_left > eye_closed_duration_threshold or elapsed_time_right > eye_closed_duration_threshold:
                    cv2.putText(frame, "ALERT! Eyes Closed", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    play_sound("alarm_1.wav")

        else:
            start_time_left = None
            start_time_right = None

        
#         cv2.polylines(frame, [left_eye_points], isClosed=True, color=(0, 255, 0), thickness=2)
#         cv2.polylines(frame, [right_eye_points], isClosed=True, color=(0, 255, 0), thickness=2)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:  
        break

cap.release()
cv2.destroyAllWindows()
